In [15]:
#scp -P 8787 .\95charimg_scale_250.zip e806@163.13.136.85:/home/e806/hoiee

# Standard libraries
import re
import csv
import sys
import shutil
import os 
import math
import time
import glob

# Third-party libraries
#torch libraries
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F



from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tabula
import aircv as ac
import pyautogui
from tqdm import tqdm

# Utilities
from functools import cmp_to_key
from utils import *
import warnings

warnings.simplefilter(action='ignore', category=RuntimeWarning)


In [2]:
torch.cuda.is_available()

True

In [3]:

num_epochs = 100 #訓練的迭代次數，設為100。一個迭代代表在訓練過程中完整遍歷整個資料集一次。
batch_size = 5 #每個批次的樣本數量，用於訓練。設為128，這意味著模型會在處理每個包含128個樣本的批次後更新參數。
capacity = 64 #VAE模型中隱藏層的容量，設為64。該參數決定了隱藏層中神經元的數量。
learning_rate = 1e-3 #訓練過程中優化器使用的學習率。設為0.001（1e-3），控制優化算法的步幅大小。
variational_beta = 1 #在VAE損失函數中應用於Kullback-Leibler（KL）散度項目的權重。設為1，表示模型對重構損失和KL散度賦予相等的重要性。
use_gpu = True #一個布爾標誌，指示是否在訓練時使用GPU（如果可用）。設為True，如果有兼容的GPU存在，則啟用GPU加速。


In [4]:
x_file = "./4_1_x.npy"
y_file = "./4_1_y.npy"

# 讀取x檔案
# 讀取y檔案
x_data = np.load(x_file, allow_pickle=True).astype('float')
y_data = np.load(y_file, allow_pickle=True).astype('float')

print(x_data.shape)
print(y_data.shape)

x_data = np.transpose(x_data, (0, 1, 4, 2, 3))

print(x_data.shape)
print(y_data.shape)


train_data_x = x_data[0:40,:]
test_data_x = x_data[40:44,:]
train_data_y = y_data[0:40,:]
test_data_y = y_data[40:44,:]

print(train_data_x.shape)
print(test_data_x.shape)
print(train_data_y.shape)
print(test_data_y.shape)

# 將資料轉換為Tensor類型
train_x_tensor = torch.Tensor(train_data_x)
train_y_tensor = torch.Tensor(train_data_y)
test_x_tensor = torch.Tensor(test_data_x)
test_y_tensor = torch.Tensor(test_data_y)

# 建立訓練集和測試集的TensorDataset
train_dataset = TensorDataset(train_x_tensor,train_y_tensor)
test_dataset = TensorDataset(test_x_tensor,test_y_tensor)

# 設定批次大小
#batch_size = 2

# 建立訓練集和測試集的DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=True)


(43, 25, 100, 100, 3)
(43, 5)
(43, 25, 3, 100, 100)
(43, 5)
(40, 25, 3, 100, 100)
(3, 25, 3, 100, 100)
(40, 5)
(3, 5)


In [5]:
#x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        return x

class ModifiedMultiConvNet(nn.Module):
    def __init__(self, num_modules=25):
        super(ModifiedMultiConvNet, self).__init__()
        
        # Create a list of ConvModules
        self.modules_list = nn.ModuleList([CNN() for _ in range(num_modules)])
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(25*12*12*128, 5)
        
    def forward(self, inputs):
        assert len(inputs) == len(self.modules_list), "Number of inputs should match number of modules"
        
        outputs = []
        for i, module in enumerate(self.modules_list):
            outputs.append(module(inputs[i]))

        concatenated_output = torch.cat(outputs, dim=1)  # Concatenate along the channel dimension
        flattened_output = self.flatten(concatenated_output)  
        final_output = self.fc(flattened_output)  # Pass through the FC layer

        return final_output

# Create an instance of the modified model with 25 modules
model = ModifiedMultiConvNet().to('cuda')





In [6]:

from tqdm import tqdm

learning_rate = 1e-3

#定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()

train_loss_avg = []

print('Training ...')
for epoch in range(num_epochs):
    train_loss_avg.append(0)
    num_batches = 0

    # 使用 tqdm 包裝 train_dataloader，顯示進度條
    for x,y in tqdm(train_dataloader):
        x  = x.to("cuda")
        y  =  y.to("cuda")
        # Create dummy inputs for each of the 25 modules
        # Forward pass using the modified model
        pre_y = model([x[:,i] for i in range(25)])

        loss = criterion(pre_y,y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss_avg[-1] += loss.item()
        num_batches += 1

    train_loss_avg[-1] /= num_batches
    print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, num_epochs, train_loss_avg[-1]))

    

Training ...


100%|██████████| 20/20 [00:04<00:00,  4.84it/s]


Epoch [1 / 100] average reconstruction error: 1559731.688344


100%|██████████| 20/20 [00:00<00:00, 29.10it/s]


Epoch [2 / 100] average reconstruction error: 936.805479


...

Epoch [5 / 100] average reconstruction error: 416.101749


100%|██████████| 20/20 [00:00<00:00, 30.12it/s]


Epoch [6 / 100] average reconstruction error: 318.033396


  5%|▌         | 1/20 [00:00<00:01, 13.85it/s]


KeyboardInterrupt: 

In [16]:
#torch.save(model.state_dict(), '4_1_far4325model.pt')
model.load_state_dict(torch.load('./weight/4_1_far4325model.pt'))

<All keys matched successfully>

In [20]:
model.eval()

train_data_x = path_cropimg_to_eval(path = './337556(1).jpg') 
train_x_tensor = torch.Tensor(train_data_x)
print(train_x_tensor.shape)


x = train_x_tensor.to("cuda")
pre_y = model([x[:,i] for i in range(25)])

print(pre_y)
a= pre_y.cpu()
print(np.mean(a[0].detach().numpy()))
    

torch.Size([1, 25, 3, 100, 100])
tensor([[103.9651,  89.1274, 104.7516, 108.5049, 115.3910]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
104.34801
